# Provenance Queries: Family Relations
* This assignment uses Datalog rules, so refer to the earlier Datalog assignment before attempting this notebook. As usual, run the cells in order.  
* All Datalog cells start with `%%clingo` (we use the [clingo](https://potassco.org/clingo/), answer set programming system. It's a bit overkill for Datalog, but works :-)
* You can run your clingo cell against the facts and rules from a file. `set_db_file $filepath` sets the file against which your clingo cells will run.
* The clingo cells are _independent_ of each other. Rules defined in one cell won't be visible in others!
* When you submit the assignment, we will run your code against different sets of facts. (So don't "hardcode" your answers ;-)
* If you want to practice a bit more with clingo, you can go to https://potassco.org/clingo/run/ and experiment with the small (but interesting!) examples there. Alternatively, you can install clingo on your own computer:  https://potassco.org/doc/start. If you want to go "all in", there is also an online course (https://teaching.potassco.org/). But don't worry: _none of this is required to do this assignment!_

## Family data and rules in Datalog (clingo):
* Consider the family relation which is shown as a graph below! We will work with Datalog rules to query data from this graph and check for integrity violations.
![Ancestor](ancestor.png "Family")

In this assignment you will be using Datalog as a query language for provenance. Provenance graphs are directed, acyclic graphs (DAGs), connecting nodes (e.g., data elements, workflow steps, etc.) through directed edges. Edges typically model dataflow, e.g., _read_ and _write_ events in the case of retrospective provenance information, or _in_ and _out_ edges in a prospective provenance graph (i.e., a workflow graph).

Similarly, provenance graphs might also represent the “backward dependencies” (_used_, _wasGeneratedBy_) of the Open Provenance Model (OPM) or the W3C PROV model.

Problem 1 resumes the “family relations” theme from an earlier assignment, using relationships that are useful when querying data provenance graphs. For example, a recursively defined relation such as `anc(x,y)` (for ancestor) can be used to trace the lineage of a data product from a workflow run. 

Here, in graph G there is an edge from _X_ to _Y_, if _X_ is a _child_ of _Y_, or equivalently, if _X_ _has_parent_ _Y_. In the Datalog program _P_ this is represented by a fact `par(X,Y)`. The rules for `anc(X,Y)` define the transitive ancestor relation, i.e., `anc(X,Y)` is true if one can reach from _X_ an ancestor _Y_ via a chain of parent edges. Similarly, `ca(X,Y,A)` means that _A_ is a _common ancestor_ of _X_ and _Y_, and `lca(X,Y,A)` means that _A_ is the _least common ancestor_ (LCA) of _X_ and _Y_. To compute the LCA, we use an auxiliary relation `not_lca(X,Y,A)`, which states that _A_ is not the LCA of _X_ and _Y_, since there exists another, lower common ancestor _A1_.

In [1]:
%reload_ext lib.clingo.clingo_magic
import os
from lib.clingo.clingo_evaluate_util import clingo_evaluate

In [2]:
# All clingo cells will run against this file containing the base facts.
family_base_facts_and_rules_file = os.path.expanduser('~/data_readonly/provenance/problem1-lca.lp')
%set_db_file $family_base_facts_and_rules_file

'% The parent facts par(Child, Parent)\npar(albert,bernhard). \npar(albert,monique).\npar(bernhard,hermann). \npar(bernhard,johanna).\npar(eva,kati).\npar(franz,rita). \npar(franz,wilhelm).  \npar(heinz,rita). \npar(heinz,wilhelm).\npar(hermann,rita).\npar(hermann,wilhelm). \npar(karl,hermann). \npar(karl,johanna). \npar(kati,hermann).\npar(kati,johanna).'

### [5 points] 1. Albert's ancestors
Write a Datalog rule for finding the ancestors of _albert_.

In [3]:
%%clingo {"predicate" : "ancestor_albert", "predicate_arity" : 1, "result_var": "Ancestor_albert"}
% Don't change the clingo magic command above. The header of this cell will determine how the datalog rules are saved for evaluation.

% Change the following expression:

anc(X,Y) :- par(X,Y).
anc(X,Y) :- par(X,Z), anc(Z,Y).

ancestor_albert(X) :- anc(albert, X).

Saving output to local variable Ancestor_albert['result']
Saving code snippet to local variable Ancestor_albert['code']


'ancestor_albert(bernhard) ancestor_albert(monique) ancestor_albert(johanna) ancestor_albert(hermann) ancestor_albert(rita) ancestor_albert(wilhelm)'


#### Test 1 for Albert's ancestors
The following test will compare the output of your `ancestor_albert` rule against the expected output.
You must have run all clingo cells above for test to pass.

In [4]:
# The following should be the output of your previous cell.
# The order of predicates in the output doesn't matter. 
# Run this cell to see the expected output with syntax highlighting.
expected_output = '''ancestor_albert(bernhard) ancestor_albert(monique) ancestor_albert(johanna) ancestor_albert(hermann) ancestor_albert(rita) ancestor_albert(wilhelm)'''
db_file = os.path.expanduser('~/data_readonly/provenance/problem1-lca.lp')
clingo_evaluate(db_file, Ancestor_albert['code'], 'ancestor_albert', 1, expected_output)

'ancestor_albert(bernhard) ancestor_albert(monique) ancestor_albert(johanna) ancestor_albert(hermann) ancestor_albert(rita) ancestor_albert(wilhelm)'

In [5]:
# Hidden Test for ancestor_albert/1
# The following is a hidden test case. 
# As before: this will always pass in your version and will only be evaluated after submission.



### [5 points] 2. Eva's ancestors
Write a Datalog rule for finding the ancestors of _eva_.

In [6]:
%%clingo {"predicate" : "ancestor_eva", "predicate_arity" : 1, "result_var": "Ancestor_eva"}
% Don't change the clingo magic command above. The header of this cell will determine how the datalog rules are saved for evaluation.

% Change the following expressions:
anc(X,Y) :- par(X,Y). % Base Case
anc(X,Y) :- par(X,Z), anc(Z,Y). % Recursive Case

ancestor_eva(X) :- anc(eva, X).

Saving output to local variable Ancestor_eva['result']
Saving code snippet to local variable Ancestor_eva['code']


'ancestor_eva(kati) ancestor_eva(johanna) ancestor_eva(hermann) ancestor_eva(rita) ancestor_eva(wilhelm)'

#### Test 2 for Eva's ancestors
The following test will compare the output of your `ancestor_eva` rule(s) against the expected output. You must have run all clingo cells above for test to pass.


In [7]:
# The following should be the output of your previous cell.
# The order of predicates in the output doesn't matter. 
# Run this cell to see the expected output with syntax highlighting.

expected_output = '''ancestor_eva(kati) ancestor_eva(johanna) ancestor_eva(hermann) ancestor_eva(rita) ancestor_eva(wilhelm)'''
db_file = os.path.expanduser('~/data_readonly/provenance/problem1-lca.lp')
clingo_evaluate(db_file, Ancestor_eva['code'], 'ancestor_eva', 1, expected_output)

'ancestor_eva(kati) ancestor_eva(johanna) ancestor_eva(hermann) ancestor_eva(rita) ancestor_eva(wilhelm)'

In [8]:
# Hidden test case for ancestor_eva/1
# The following is a hidden test case. 
# As before: this will always pass in your version and will only be evaluated after submission.




### [6 points] 3. Common Ancestors of Albert and Eva
Write a Datalog rule for finding the common ancestors of _albert_ and _eva_.

In [9]:
%%clingo {"predicate" : "ca_albert_eva", "predicate_arity" : 1, "result_var": "Ca_albert_eva"}
% Don't change the clingo magic command above. The header of this cell will determine how the datalog rules are saved for evaluation.

% Change the following expressions (you might need multiple rules for some relations)
anc(X,Y) :- par(X,Y).
anc(X,Y) :- par(X,Z), anc(Z,Y).

ca(X,A,A) :- anc(X,A).
ca(A,X,A) :- anc(X,A).
    
ca(X,Y,A) :- anc(X,A), anc(Y,A), X != Y. 

ca_albert_eva(X) :- ca(albert, eva, X).

Saving output to local variable Ca_albert_eva['result']
Saving code snippet to local variable Ca_albert_eva['code']


'ca_albert_eva(johanna) ca_albert_eva(hermann) ca_albert_eva(rita) ca_albert_eva(wilhelm)'

#### Test 3 for common Ancestors of Albert and Eva.

In [10]:
expected_output = '''ca_albert_eva(johanna) ca_albert_eva(hermann) ca_albert_eva(rita) ca_albert_eva(wilhelm)'''

db_file = os.path.expanduser('~/data_readonly/provenance/problem1-lca.lp')
clingo_evaluate(db_file, Ca_albert_eva['code'], 'ca_albert_eva', 1, expected_output)


'ca_albert_eva(johanna) ca_albert_eva(hermann) ca_albert_eva(rita) ca_albert_eva(wilhelm)'

In [11]:
# The following is a hidden test case. 
# As before: this will always pass in your version and will only be evaluated after submission.


### [12 points] 4. Which of the common ancestors in (3) are LCAs?
Write Datalog rules for the **lowest** common ancestors for _albert_ and _eva_.

In [12]:
%%clingo {"predicate" : "lca_albert_eva", "predicate_arity" : 1, "result_var": "Lca_albert_eva"}
% Don't change the clingo magic command above. The header of this cell will determine how the datalog rules are saved for evaluation.

% Change the following expressions:
anc(X,Y) :- par(X,Y).
anc(X,Y) :- par(X,Z), anc(Z,Y).

ca(X,A,A) :- anc(X,A).
ca(A,X,A) :- anc(X,A).
    
ca(X,Y,A) :- anc(X,A), anc(Y,A), X != Y.  

% Provide rules for not_lca
not_lca(X,Y,A) :- ca(X,Y,A), ca(X,Y,A1), anc(A1,A).

lca(X,Y,A) :- ca(X,Y,A), not not_lca(X,Y,A).

lca_albert_eva(X) :- lca(albert,eva,X).

Saving output to local variable Lca_albert_eva['result']
Saving code snippet to local variable Lca_albert_eva['code']


'lca_albert_eva(johanna) lca_albert_eva(hermann)'


#### Test 4 for LCA

In [13]:
expected_output = '''
lca_albert_eva(johanna) lca_albert_eva(hermann)
'''
db_file = os.path.expanduser('~/data_readonly/provenance/problem1-lca.lp')
clingo_evaluate(db_file, Lca_albert_eva['code'], 'lca_albert_eva', 1, expected_output)

'lca_albert_eva(johanna) lca_albert_eva(hermann)'

### [12 points] 5. Which of the common ancestors in (3) are _not_ LCAs? 
Write Datalog rules for the common ancestor of _albert_ and _eva_ that are **not** LCAs.

In [15]:
%%clingo {"predicate" : "nlca_albert_eva", "predicate_arity" : 1, "result_var": "Nlca_albert_eva"}
% Don't change the clingo magic command above. The header of this cell will determine how the datalog rules are saved for evaluation.

% Change following expression.
anc(X,Y) :- par(X,Y).
anc(X,Y) :- par(X,Z), anc(Z,Y).

ca(X,A,A) :- anc(X,A).
ca(A,X,A) :- anc(X,A).
    
ca(X,Y,A) :- anc(X,A), anc(Y,A), X != Y.  

% provide negative rules for not lca
not_lca(X,Y,A) :- ca(X,Y,A), ca(X,Y,A1), anc(A1,A).

lca(X,Y,A) :- ca(X,Y,A), not not_lca(X,Y,A).

nlca_albert_eva(X) :- not_lca(albert,eva,X).

Saving output to local variable Nlca_albert_eva['result']
Saving code snippet to local variable Nlca_albert_eva['code']


'nlca_albert_eva(rita) nlca_albert_eva(wilhelm)'

#### Test 5 for common ancestor but not lowest common ancestor

In [16]:
expected_output = '''
nlca_albert_eva(rita) nlca_albert_eva(wilhelm)
'''
db_file = os.path.expanduser('~/data_readonly/provenance/problem1-lca.lp')
clingo_evaluate(db_file, Nlca_albert_eva['code'], 'nlca_albert_eva', 1, expected_output)

'nlca_albert_eva(rita) nlca_albert_eva(wilhelm)'